# Задача. Функция распределения пользователей

Реализуйте функцию `process_user`



In [1]:
import hashlib

import numpy as np
import pandas as pd


def get_hash_modulo(value: str, modulo: int, salt: str):
    """Вычисляем остаток от деления: (hash(value + salt)) % modulo."""
    hash_value = int(hashlib.md5(str.encode(value + salt)).hexdigest(), 16)
    return hash_value % modulo

def process_user(user_id, buckets, experiments, bucket_salt):
    """Определяет в какие эксперименты попадает пользователь.

    Сначала нужно определить бакет пользователя.
    Затем для каждого эксперимента в этом бакете выбрать пилотную или контрольную группу.

    :param user_id (str): идентификатор пользователя
    :param buckets (list[list[int]]): список бакетов, в каждом бакете перечислены
            идентификаторы экспериментов, которые в нём проводятся.
    :param experiments (list[dict]): список словарей с информацией об экспериментах.
        Ключи словарей:
        - id (int) - идентификатор эксперимента
        - salt (str) - соль эксперимента для распределения пользователей на
            контрольную/пилотную группы.
    :param bucket_salt (str): соль для разбиения пользователей по бакетам.
        При одной соли каждый пользователь должен всегда попадать в один и тот же бакет.
        Если изменить соль, то распределение людей по бакетам должно измениться.
    :return bucket_id, experiment_groups:
        - bucket_id (int) - номер бакета (индекс элемента в buckets)
        - experiment_groups (list[tuple]) - список пар: id эксперимента, группа.
            Группы: 'A', 'B'.
        Пример: (8, [(194, 'A'), (73, 'B')])
    """
    bucket_id = get_hash_modulo(user_id, len(buckets), bucket_salt)
    exp_ids = set(buckets[bucket_id])
    experiment_groups = []
    for exp_data in experiments:
        if exp_data['id'] in exp_ids:
            experiment_groups.append((exp_data['id'], ['A', 'B'][get_hash_modulo(user_id, 2, exp_data['salt'])]))
    return bucket_id, experiment_groups

In [2]:
# тесты

experiment_ids = [2, 7, 9]
experiments = [{'id': x, 'salt': str(x)} for x in experiment_ids]
buckets = [[2, 7], [], [9], [7]]
buckets_count = len(buckets)
bucket_salt = 'dfs6&6dap_!4'
n_users = 1000
user_ids = [str(x) for x in range(n_users)]
user_bucket_groups_list = []
for user_id in user_ids:
    bucket_id, experiment_groups = process_user(
        user_id, buckets, experiments, bucket_salt
    )
    user_bucket_groups_list.append((bucket_id, experiment_groups,))
    assert bucket_id in range(buckets_count)
    assert len(experiment_groups) == len(buckets[bucket_id])
    for exp_id, group in experiment_groups:
        assert exp_id in experiment_ids
        assert group in ['A', 'B']
print('Формат вывода. Тест пройден')

buckets_ = [x[0] for x in user_bucket_groups_list]
buckets_weiths = pd.Series(buckets_).value_counts(normalize=True)
max_delta = np.max(np.abs((buckets_weiths - 1 / buckets_count).values))
# разрешаем отклонение в 4 сигма
bound_delta = 4 * ((1 / buckets_count * (1 - 1 / buckets_count) / n_users) ** 0.5)
assert max_delta < bound_delta
print('Равномерность распределения по бакетам. Тест пройден')

user_groups_list = [sorted(x[1]) for x in user_bucket_groups_list]
groups_ = [' '.join([' '.join(map(str, y)) for y in x]) for x in user_groups_list]
groups_weiths = pd.Series(groups_).value_counts(normalize=True)
w_ = pd.Series({
    '2 A 7 A': 1,
    '2 A 7 B': 1,
    '2 B 7 A': 1,
    '2 B 7 B': 1,
    '7 A': 2,
    '7 B': 2,
    '9 A': 2,
    '9 B': 2,
    '': 4
})
w = w_ / w_.sum()
df_w = pd.merge(
    pd.DataFrame({'ideal': w}),
    pd.DataFrame({'res': groups_weiths}),
    how='left', left_index=True, right_index=True
).fillna(0)
df_w['delta'] = (df_w['res'] - df_w['ideal']).abs()
# разрешаем отклонение в 4 сигма
df_w['bound_delta'] = 4 * ((w * (1 - w) / n_users) ** 0.5)
assert (df_w['delta'] < df_w['bound_delta']).all()
print('Равномерность распределения по группам. Тест пройден')

for user_id, user_bucket_groups in zip(user_ids, user_bucket_groups_list):
    (prev_bucket_id, prev_experiment_groups) = user_bucket_groups
    bucket_id, experiment_groups = process_user(
        user_id, buckets, experiments, bucket_salt
    )
    assert prev_bucket_id == bucket_id
    str_experiment_groups = ' '.join(
        [' '.join(map(str, y)) for y in sorted(experiment_groups)]
    )
    str_prev_experiment_groups = ' '.join(
        [' '.join(map(str, y)) for y in sorted(prev_experiment_groups)]
    )
    assert str_experiment_groups == str_prev_experiment_groups        
print('Воспроизводимость результатов для одной соли. Тест пройден')

list_equals = []
for user_id, (prev_bucket_id, _) in zip(user_ids, user_bucket_groups_list):
    bucket_id, _ = process_user(user_id, buckets, experiments, bucket_salt * 2)
    list_equals.append(bucket_id == prev_bucket_id)
part_same_bucket = np.mean(list_equals)
assert part_same_bucket < 2 / buckets_count
print('Изменение распределения при замене соли. Тест пройден')

Формат вывода. Тест пройден
Равномерность распределения по бакетам. Тест пройден
Равномерность распределения по группам. Тест пройден
Воспроизводимость результатов для одной соли. Тест пройден
Изменение распределения при замене соли. Тест пройден
